In [1]:
#| default_exp interface_pymeshlab

In [266]:
#| export

from blender_tissue_cartography import io as tcio
import numpy as np
import pymeshlab

In [267]:
from importlib import reload

In [268]:
np.set_printoptions(suppress=True)

## Interfacing with `open3d`

In this notebook, we define functions to convert our `ObjMesh` class to and from `open3d`.

Note: `open3d` uses its own array-like datatype, `o3d.core.Tensor` (for autodiff-reasons I think). This means there are also _two_ triangular mesh classes, one `Tensor`-based, one not. We'll convert to the `Tensor`-based one since it appears to be the one which will be supported in the future. 

In [78]:
reload(tcio)

<module 'blender_tissue_cartography.io' from '/home/nikolas/Documents/UCSB/streichan/numerics/code/python code/jupyter notebooks/blender-tissue-cartography/blender_tissue_cartography/io.py'>

In [79]:
mesh_fname_data = "registration_example/Drosophila_CAAX-mCherry_mesh_remeshed.obj"
mesh_fname_ref = "registration_example/Drosophila_reference_preregistered.obj"

In [100]:
mesh_data = tcio.ObjMesh.read_obj(mesh_fname_data)
mesh_ref = tcio.ObjMesh.read_obj(mesh_fname_ref)

mesh_data.match_vertex_info(require_texture_normals=False)
mesh_ref.match_vertex_info(require_texture_normals=False)

In [237]:
#| export

def convert_to_pymeshlab(mesh: tcio.ObjMesh) -> pymeshlab.Mesh:
    """
    Convert tcio.ObjMesh to pymeshlab.Mesh.
    
    See https://pymeshlab.readthedocs.io/en/latest/classes/mesh.html
    Note: normal information is recalculated by pymeshlab. Discards any non-triangle faces.
    """
    mesh.match_vertex_info(require_texture_normals=False)
    if np.isnan(mesh.matched_texture_vertices).any():
        return pymeshlab.Mesh(vertex_matrix=mesh.matched_vertices, face_matrix=mesh.tris)
    return pymeshlab.Mesh(vertex_matrix=mesh.matched_vertices, face_matrix=mesh.tris,
                          v_tex_coords_matrix=mesh.matched_texture_vertices.astype(np.float64))

In [238]:
pymesh_data = convert_to_pymeshlab(mesh_data)
pymesh_ref = convert_to_pymeshlab(mesh_ref)

In [239]:
np.allclose(mesh_ref.matched_vertices, pymesh_ref.vertex_matrix())

True

In [240]:
pymesh_normals = pymesh_ref.vertex_normal_matrix()
pymesh_normals = (pymesh_normals.T / np.linalg.norm(pymesh_normals, axis=-1)).T

np.einsum('vi,vi->v', mesh_ref.matched_normals, pymesh_normals)

array([0.99904184, 0.99822658, 0.99668032, ..., 0.99417082, 0.99599803,
       0.99721726])

In [243]:
np.allclose(mesh_ref.matched_texture_vertices, pymesh_ref.vertex_tex_coord_matrix())

True

In [244]:
%%timeit
convert_to_pymeshlab(mesh_ref)

297 ms ± 3.66 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [263]:
#| export

def convert_from_pymeshlab(mesh: pymeshlab.pmeshlab.Mesh) -> pymeshlab.Mesh:
    """Convert pymeshlab mesh to ObjMesh."""
    vertices = mesh.vertex_matrix()
    faces = [[3*[v,] for v in f] for f in pymesh_ref.face_matrix()]
    normals = pymesh_ref.vertex_normal_matrix()
    normals = (normals.T / np.linalg.norm(normals, axis=-1)).T
    if mesh.has_vertex_tex_coord():
        return tcio.ObjMesh(vertices=vertices, faces=faces, normals=normals,
                            texture_vertices=mesh.vertex_tex_coord_matrix())
    return tcio.ObjMesh(vertices=vertices, faces=faces, normals=normals)

In [264]:
mesh_reconverted = convert_from_pymeshlab(pymesh_ref)

In [265]:
mesh_reconverted.texture_vertices

array([[0.84753603, 0.99985301],
       [0.84363198, 0.99985301],
       [0.83972698, 0.99985403],
       ...,
       [0.007827  , 0.000135  ],
       [0.003923  , 0.000135  ],
       [0.000018  , 0.000136  ]])